In [1]:
# Create folder named experiment#1 in processed data, outputs and analysis

# Get data for flow, tessellations, demographics and geographical boundary of WA

# Split data into training and test - create four files: Flows for train and test, region indices for train and test
# Save these files in processed data folder 
# TODO Create visualization and save to outputs of experiement

# Biased sampling on the training data to create two biased datasets - save in processed data folder for experiment1 

# Run single constrained gravity model on biased training data and original data to create three models 
# Run trained models on unbiased test data and create generated flows in outputs 

# Use generated flows from outputs and test true labels from processed data to create fairness metrics and visualizations
# Save this in outputs - fairness.json and fairness.png (for each model)

# USE ML TUTORIAL ENVN

In [1]:
import sys
import os
import pandas as pd
import matplotlib.pyplot as plt

sys.path.append(os.path.abspath('../preprocessing'))
from train_test_processing import * 
sys.path.append(os.path.abspath('../preprocessing'))
from train_test_split_vis import * 

experiment_id = '1'

# washington = load_state_or_county_data('../data/WA/boundary.geojson')
washington = load_state_or_county_data('../data/WA/WA_State_Boundary.geojson')
flow_df = pd.read_csv('../data/WA/flow.csv')
features_df = pd.read_csv('../data/WA/features.csv')
tessellation_df = load_state_or_county_data('../data/WA/tessellation_wpop.geojson')
# grid = create_grid(washington.unary_union, 25, washington.crs)
grid = create_grid(washington.unary_union, 25)

# Split the data into train and test sets
train_output, test_output = flow_train_test_split(tessellation_df, features_df, grid, experiment_id = experiment_id)

# train_geoids = set(train_output['census_tracts_geoids'].explode())
# print(len(train_geoids))

# vis
# plot_grid_and_census_tracts(grid, tessellation_df, train_output, test_output)

filter_train_test_data(flow_df, tessellation_df, features_df, train_output, test_output, experiment_id = experiment_id)




/Users/apoorvasheera/anaconda3/envs/MLTutorial/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3505: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


ValueError: 'left_df' should be GeoDataFrame, got <class 'pandas.core.frame.DataFrame'>

In [1]:
import sys
import os
import pandas as pd
import matplotlib.pyplot as plt

experiment_id = "1"

sys.path.append(os.path.abspath('../preprocessing'))
from biased_sampling import *

features_df = pd.read_csv("../processed_data/1/train/train_features.csv")
train_flows_df = pd.read_csv("../processed_data/1/train/flows/train_flow.csv")
demographics_df = pd.read_csv("../data/WA/demographics.csv")


calculate_biased_flow(features_df, demographics_df, train_flows_df, demographic_column_name='svi', method=1, order="ascending", sampling=False, experiment_id=experiment_id, bias_factor=0.5)
calculate_biased_flow(features_df, demographics_df, train_flows_df, demographic_column_name='svi', method=1, order="descending", sampling=False, experiment_id=experiment_id, bias_factor=0.5)


/Users/apoorvasheera/Documents/DSSG/Crowd Flow/crowdflow-fairness/preprocessing/biased_sampling.py:28: FutureWarning: Passing 'suffixes' which cause duplicate columns {'geoid_x'} in the result is deprecated and will raise a MergeError in a future version.
  destination_merge = pd.merge(destination_merge, features[['geoid', 'total_population']], how='left', left_on='destination', right_on='geoid')


Columns with missing data and their count:
origin               0
destination          0
demographic_o    14031
demographic_d    15057
population_o     14031
population_d     15057
flow                 0
dtype: int64
Missing data diagnostic saved to ../outputs/1/train/missing_data_diagnosis.csv
(30023, 7)
Saved adjusted flows to ../processed_data/1/train/flows/svi/1_ascending_biased_flow.csv
Columns with missing data and their count:
origin               0
destination          0
demographic_o    14031
demographic_d    15057
population_o     14031
population_d     15057
flow                 0
dtype: int64
Missing data diagnostic saved to ../outputs/1/train/missing_data_diagnosis.csv
(30023, 7)


/Users/apoorvasheera/Documents/DSSG/Crowd Flow/crowdflow-fairness/preprocessing/biased_sampling.py:28: FutureWarning: Passing 'suffixes' which cause duplicate columns {'geoid_x'} in the result is deprecated and will raise a MergeError in a future version.
  destination_merge = pd.merge(destination_merge, features[['geoid', 'total_population']], how='left', left_on='destination', right_on='geoid')


Saved adjusted flows to ../processed_data/1/train/flows/svi/1_descending_biased_flow.csv


In [ ]:
# Here change environement to one with scikit-mobility

In [ ]:
import sys
import os
sys.path.append(os.path.abspath('../models'))
from gravity import *

experiment_id = "1"

tessellation_train = gpd.read_file("../processed_data/1/train/train_tessellation.geojson")
tessellation_test = gpd.read_file("../processed_data/1/test/test_tessellation.geojson")

gravity = grav_Model(tessellation_train, tessellation_test, "../processed_data/1/train/flows/train_flow.csv","../processed_data/1/test/flows/test_flow.csv", "gravity_singly_constrained", 'flows', experiment_id=experiment_id)






In [ ]:
tracts = gpd.read_file()

In [2]:
tessellation_train = gpd.read_file("../processed_data/1/train/train_tessellation.geojson")
tessellation_test = gpd.read_file("../processed_data/1/test/test_tessellation.geojson")

tessellation_train

,GEOID,XCOORD,YCOORD,total_population,geometry
0,53003960100,2.478878e+06,316229.093061,4320,"POLYGON ((2407498.76364 263743.17398, 2407478...."
1,53003960300,2.510151e+06,414911.963972,3633,"POLYGON ((2504540.37717 416766.54412, 2504886...."
2,53003960400,2.505962e+06,412445.800829,2517,"POLYGON ((2502838.27233 412941.39943, 2502910...."
3,53003960500,2.510481e+06,410373.766615,3517,"POLYGON ((2507857.92970 408364.67024, 2507893...."
4,53003960600,2.507962e+06,405277.944477,3894,"POLYGON ((2502417.95942 404700.62798, 2502406...."
...,...,...,...,...,...
576,53077002101,1.757987e+06,396967.615952,2297,"POLYGON ((1716307.79591 387056.62897, 1716307...."
577,53077002801,1.585170e+06,449477.275913,5865,"POLYGON ((1559608.17932 441157.52938, 1559611...."
578,53077940002,1.671599e+06,376587.366439,4731,"POLYGON ((1634622.09067 368811.57790, 1634623...."
579,53077940005,1.686814e+06,379232.681614,4727,"POLYGON ((1681785.22522 381158.56677, 1681814...."


In [3]:
tessellation_test

,GEOID,XCOORD,YCOORD,total_population,geometry
0,53001950100,2.164853e+06,671104.074676,2606,"POLYGON ((2018032.07503 696360.32890, 2018022...."
1,53001950200,2.138151e+06,585491.984120,1763,"POLYGON ((2019612.57190 579263.24944, 2019614...."
2,53001950400,1.974047e+06,549486.692609,3144,"POLYGON ((1970740.91951 547174.93228, 1970739...."
3,53001950500,1.975942e+06,541548.994721,5660,"POLYGON ((1968357.47234 536445.67382, 1968376...."
4,53003960200,2.456469e+06,391335.882695,4755,"POLYGON ((2417940.12672 381783.84509, 2417932...."
...,...,...,...,...,...
569,53077003100,1.619895e+06,500397.085193,5151,"POLYGON ((1594373.54961 510882.28934, 1594425...."
570,53077003200,1.630376e+06,478912.391508,6851,"POLYGON ((1624779.14869 474582.53222, 1624781...."
571,53077003400,1.622740e+06,482230.093423,4934,"POLYGON ((1611534.01258 483165.92967, 1611581...."
572,53077940001,1.625990e+06,393922.733054,6289,"POLYGON ((1592154.02354 406573.50909, 1592189...."


In [3]:
# Load tessellation
tessellation_train = gpd.read_file("../processed_data/1/train/train_tessellation.geojson")
tessellation_test = gpd.read_file("../processed_data/1/test/test_tessellation.geojson")

# Load flow data
train_flow = pd.read_csv("../processed_data/1/train/flows/train_flow.csv")
test_flow = pd.read_csv("../processed_data/1/test/flows/test_flow.csv")

# Extract unique geoids from flow data
train_geoids = set(train_flow['origin'].unique()).union(set(train_flow['destination'].unique()))
test_geoids = set(test_flow['origin'].unique()).union(set(test_flow['destination'].unique()))

# Extract unique geoids from tessellation data
tessellation_train_geoids = set(tessellation_train['GEOID'].unique())
tessellation_test_geoids = set(tessellation_test['GEOID'].unique())

# Identify missing geoids
missing_train_geoids = list(train_geoids - tessellation_train_geoids)
missing_test_geoids = list(test_geoids - tessellation_test_geoids)

# Combine missing geoids and identify if from train or test
missing_geoids_df = pd.DataFrame({
    'GEOID': missing_train_geoids + missing_test_geoids,
    'Dataset': ['Train'] * len(missing_train_geoids) + ['Test'] * len(missing_test_geoids)
})

# Save to CSV
output_path = "../processed_data/1/missing_geoids_tessellation.csv"
missing_geoids_df.to_csv(output_path, index=False)
print(f"Missing geoids have been saved to {output_path}.")

Missing geoids have been saved to ../processed_data/1/missing_geoids_tessellation.csv.


In [7]:
import os
flow_file_location = "../processed_data/1/train/flows/train_flows.csv"
path_parts = flow_file_location.split(os.sep)
filename = path_parts[-1]

flow_dir_index = path_parts.index('flows') + 1
subdirectory = os.sep.join(path_parts[flow_dir_index:-1])
if subdirectory:
    # Create output filename for structured subdirectory cases
    new_filename = filename.replace('.csv', '_synthetic_flows.csv')
    output_path = os.path.join('..', 'outputs', experiment_id, subdirectory, new_filename)
else:
    # Create output filename for general cases without subdirectory
    output_path = os.path.join('..', 'outputs', experiment_id, 'synthetic_flows.csv')


# # Example usage:
# flow_path_1 = '../processed_data/experiment_1/train/flows/svi/1_ascending_biased_flows.csv'
# flow_path_2 = '../processed_data/experiment_1/train/flows/train_flows.csv'

# output_path_1 = create_output_path(flow_path_1)
# output_path_2 = create_output_path(flow_path_2)

print("Output path for flow_path:", output_path)


Output path for flow_path: ../outputs/1/synthetic_flows.csv
